In [12]:
library(ggplot2)
library(ggrepel)
library(EMCluster)
library(ggpmisc)
library(mclust)

In [13]:
data_profile <- read.csv('/Users/m221138/RA_project/analysis/linear_model_r1/plots/clp/clp.plot.profile.tsv', sep="\t", row.names=1, header=TRUE)
data_profile <- as.data.frame(t(data_profile))


In [15]:

NUM_CHEM_ID <- ncol(data_profile)
NUM_patient_timepoint <- nrow(data_profile)

print (NUM_CHEM_ID)
print (NUM_patient_timepoint)

significant_chemIDs = ''

for (i in 1:NUM_CHEM_ID){
    if (i > 2){
        
        chemID <- colnames(data_profile)[i]
        print (chemID)
        output_name <- paste('/Users/m221138/RA_project/analysis/linear_model_r1/plots/clp/',chemID, ".pdf", sep="")


        chem_profile <- data_profile[,i]
        simple_LRM <- lm(DAS28 ~ chem_profile, data=data_profile)

        bad <- subset(data_profile, data_profile$DAS_LABEL == 1, select = c("DAS28",chemID,"DAS_LABEL"))
        good <- subset(data_profile, data_profile$DAS_LABEL == 0, select = c("DAS28",chemID,"DAS_LABEL"))
        bad_lm <- lm(DAS28 ~ bad[,2], data=bad)
        bad_lm_stats <- summary(bad_lm)$fstatistic
        bad_lm_f_pvalue<- pf(bad_lm_stats[1], bad_lm_stats[2], bad_lm_stats[3], lower.tail=FALSE)
        bad_value <- bad[,2]
        
        good_lm <- lm(DAS28 ~ good[,2], data=good)
        good_lm_stats <- summary(good_lm)$fstatistic
        good_lm_f_pvalue<- pf(good_lm_stats[1], good_lm_stats[2], good_lm_stats[3], lower.tail=FALSE)
        good_value <- good[,2]
        good_vs_bad <- wilcox.test(bad_value, good_value) 
        
        if (good_vs_bad$p.value < 0.05){
            temp_string <- paste(chemID, "\t", good_vs_bad$p.value, "\n", sep="")
            significant_chemIDs <- paste(significant_chemIDs, "\t", temp_string, sep ="")
        }
        
        pdf(output_name)
        result_plot <- ggplot(data_profile, aes(x = chem_profile, y = DAS28, label=DAS_LABEL)) + 
        geom_point(color = dplyr::case_when(data_profile$DAS_LABEL > 0 ~ "#bd2615", data_profile$DAS_LABEL < 1 ~ "#1585bd"), size = 3, alpha = 0.8) +
        geom_smooth(method="lm", se=FALSE, aes(colour = "Single Linear Model")) +
        geom_smooth(method="lm", data=bad, aes(x=bad[,2], y = DAS28, colour = "No Remission specific LM"),se=FALSE)+
        geom_smooth(method="lm", data=good, aes(x=good[,2], y = DAS28, colour = "Remission specific LM"), se=FALSE) +
        scale_colour_manual(name="legend", values=c("red", "blue", "black")) +
        xlab(chemID)
        print (result_plot)
        dev.off()
        
#         output_name <- paste('/Users/m221138/RA_project/analysis/linear_model_r1/plots/clp/',chemID, ".gm.pdf", sep="")
#         pdf(output_name)
#         temp_df <- data.frame(chem_profile, data_profile$DAS28)
#         ret <- Mclust(temp_df)
#         result_plot <- plot(ret, what = c("classification"))
#         print (result_plot)
#         dev.off()
    }
}

writeLines(significant_chemIDs, '/Users/m221138/RA_project/analysis/linear_model_r1/plots/clp/significant_chemID.txt')

[1] 12
[1] 118
[1] "100009616"
[1] "100009435"
[1] "100010023"
[1] "100010479"
[1] "100009628"
[1] "100010827"
[1] "100010589"
[1] "100009850"
[1] "100010220"
[1] "100009835"
